In [64]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
from moffragmentor import MOF
from moffragmentor.fragmentor.solventlocator import get_floating_solvent_molecules, get_all_bound_solvent_molecules
from moffragmentor.fragmentor.nodelocator import find_node_clusters, create_node_collection
from moffragmentor.fragmentor.linkerlocator import create_linker_collection
from moffragmentor.utils import visualize_part, _get_metal_sublist, unwrap
from moffragmentor.fragmentor.filter import point_in_mol_coords

In [70]:
mof = MOF.from_cif('/Users/kevinmaikjablonka/git/kjappelbaum/moffragmentor/tests/test_files/porphyrin_mof.cif')

/Users/kevinmaikjablonka/miniconda3/envs/moffragmentor/lib/python3.8/site-packages/pymatgen/io/cif.py:705: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/Users/kevinmaikjablonka/miniconda3/envs/moffragmentor/lib/python3.8/site-packages/pymatgen/io/cif.py:1160: UserWarning:

Issues encountered while parsing CIF: No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.



In [9]:
unbound_solvent = get_floating_solvent_molecules(mof)
    # Find nodes
node_result = find_node_clusters(mof, unbound_solvent.indices)
node_collection = create_node_collection(mof, node_result)
    # Find bound solvent
    # 
bound_solvent = get_all_bound_solvent_molecules(mof, node_result.nodes)
    # Filter the linkers (valid linkers have at least two branch points)
linker_collection, edge_dict = create_linker_collection(
        mof, node_result, node_collection, unbound_solvent
    )

In [15]:
metal_in_node = _get_metal_sublist(node_result.nodes[0], mof.metal_indices)

In [61]:
from scipy.optimize import linprog

def in_hull(points, x):
    n_points = len(points)
    n_dim = len(x)
    c = np.zeros(n_points)
    A = np.r_[points.T,np.ones((1,n_points))]
    b = np.r_[x, np.ones(1)]
    lp = linprog(c, A_eq=A, b_eq=b)
    return lp.success

In [63]:
in_hull( unwrap(mof.cart_coords[linker_collection[0]._original_indices], mof.lattice), mof.cart_coords[metal_in_node[0]])

False

In [58]:
point_in_mol_coords(mof.cart_coords[metal_in_node[0]], 
    unwrap(mof.cart_coords[linker_collection[0]._original_indices], mof.lattice), mof.lattice)

True

In [45]:
import numpy as np
import pandas as pd

coords = np.vstack([mof.cart_coords[metal_in_node[0]].reshape(1,3),  unwrap(mof.cart_coords[linker_collection[0]._original_indices], mof.lattice) ] )

In [46]:
df = pd.DataFrame(data=coords, columns=['x', 'y', 'z'])

In [47]:
df['color'] = ['red'] + ['blue'] *(len(df) -1)

In [48]:
import plotly.express as px

fig = px.scatter_3d(df, x='x', y='y', z='z',
              color='color', opacity=0.7)
fig

In [31]:
from pymatgen.core import Structure

sites = []
for index in linker_collection[0]._original_indices + [metal_in_node[0]]:
    sites.append(mof.structure[index])

s = Structure.from_sites(sites)
s.to('cif', 'mudon_linker.cif')

In [75]:
bbs = mof.fragment()

2022-04-14 08:33:54.461 | INFO     | moffragmentor.fragmentor:run_fragmentation:45 - Checking for metal in linker
2022-04-14 08:33:54.484 | INFO     | moffragmentor.fragmentor:run_fragmentation:81 - Re-running fragmentation with filtered nodes (metal in linker found)
2022-04-14 08:33:56.184 | INFO     | moffragmentor.fragmentor:run_fragmentation:93 - Constructing the embedding


In [77]:
bbs.linkers[0].show_molecule()

NGLWidget()